In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models
from joblib import dump
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report


### **Data Upload:**

In [15]:
df = pd.read_csv('../data/Cleaned_Churn_modelling.csv')

In [16]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42.0,2,0.00,1,1.0,1.0,101348.88,1,1,0,0
1,608,0,41.0,1,83807.86,1,0.0,1.0,112542.58,0,0,0,1
2,502,0,42.0,8,159660.80,3,1.0,0.0,113931.57,1,1,0,0
3,699,0,39.0,1,0.00,2,0.0,0.0,93826.63,0,1,0,0
4,645,1,44.0,8,113755.78,2,1.0,0.0,149756.71,1,0,0,1


### **Preprocessing:**

In [17]:
# Assigning the independent variables to the X and the dependent variables to y.
x= df.drop('Exited',axis=1) # Drops column "Exited"and stores all the other columms in a table
y= df['Exited'] # Stores the values of the "Exited" column in a table.

# Splits the data into 80% training data and 20% testing data
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Scale the x data
scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### **Build Model**

In [18]:
ANN = models.Sequential([  #Initializes the ANN
    
    # Input layer
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)), #This input layer has 128 neurons that processes one column at a time using the activation'relu'.
    layers.Dropout(0.4),

    # Hidden layer 
    layers.Dense(units=64, activation='relu'), # Sorts through all of the probabilities using a 64 nodes.
    layers.Dropout(0.3),

    # Hidden layer 
    layers.Dense(units=32, activation='relu'), # Sorts through all of the probabilities using a 32 nodes.
    layers.Dropout(0.2),


    # Output layer
    layers.Dense(1, activation='sigmoid')  # Outputs a single sigmoid value
])

/home/jaden/projects/churn/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### **Train Model:**

In [19]:
# Compile ANN
ANN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Compute class weights and fixes any imbalance in the data
class_weights = compute_class_weight(class_weight='balanced',classes=np.unique(y_train),y=y_train) # creates a more balanced decision boundery by taking the minorty class into consideration.
class_weight_dict = dict(enumerate(class_weights))

# Fitting the training data into the ANN
history=ANN.fit(X_train, y_train, epochs=40, batch_size=32,validation_split=0.2, class_weight=class_weight_dict)

Epoch 1/40


200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6498 - loss: 0.6801 - val_accuracy: 0.6825 - val_loss: 0.5667
Epoch 2/40
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7031 - loss: 0.5793 - val_accuracy: 0.7206 - val_loss: 0.5311
Epoch 3/40
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7079 - loss: 0.5626 - val_accuracy: 0.7337 - val_loss: 0.5031
Epoch 4/40
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7217 - loss: 0.5532 - val_accuracy: 0.7588 - val_loss: 0.4682
Epoch 5/40
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7459 - loss: 0.5253 - val_accuracy: 0.7613 - val_loss: 0.4725
Epoch 6/40
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7538 - loss: 0.5139 - val_accuracy: 0.7900 - val_loss: 0.4528
Epoch 7/40
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7691 - loss: 0.4900 - val_accuracy: 0.7675 - val_loss: 0.4788
Epoch 8/40
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7671 - loss: 0.4906 - val_accuracy: 0.7912 - val_

### **Model Evaluation:**

In [20]:
loss,accuracy = ANN.evaluate(X_test,y_test)
print(f'loss = {loss:.4f}')
print(f'Accuracy = {accuracy:.4f}')


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7865 - loss: 0.4619
loss = 0.4689
Accuracy = 0.7810


In [21]:
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [22]:
ypredrun = ANN.predict(X_test)
ypredresults = (ypredrun>0.5).astype(int)

print("Classification Report:")
print(classification_report(y_test, ypredresults))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.79      0.85      1627
           1       0.45      0.74      0.56       373

    accuracy                           0.78      2000
   macro avg       0.69      0.76      0.71      2000
weighted avg       0.84      0.78      0.80      2000



In [23]:
cm = confusion_matrix(y_test, ypredresults)
cm_normalized = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(6, 4))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=['Stayed', 'Exited'], yticklabels=['Stayed', 'Exited'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Dump model to load on dash app

In [24]:
ANN.save("ann.keras")
dump(scaler, open("scaler.pkl", "wb"))